In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset

In [1]:
with open('names.txt') as f:
    content = f.read()
    words = content.splitlines()

In [3]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = { s: i+1 for i, s in enumerate(chars) }
stoi['.'] = 0
itos = { i: s for s, i in stoi.items() }
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [53]:
import torch

# Context length of how many characters support the prediction. This is consistent with
# the paper we're following
block_size = 3

x, y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        x.append(context)
        y.append(ix)
        print(''.join(itos[i] for i in context), "--->", ch)
        # Advance the rolling window of context
        context = context[1:] + [ix]


X = torch.tensor(x)
Y = torch.tensor(y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [15]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

# Embedding Lookup table

In [54]:
import torch.nn.functional as F

# C is the lookup table which is the first layer of the network
C = torch.randn((27, 2))
C.shape

torch.Size([27, 2])

In [62]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

# Hidden Layer

In [63]:
W1 = torch.randn((6, 100))
b = torch.randn(100)

In [82]:
# Hidden layer of activations for examples

# torch.concat(emb[:, 0, :], emb[:, 1, :], emb[:, 2, :])
# torch.unbind(emb, 1), 1)
h = torch.tanh(emb.view(-1, 6) @ W1 + b)
h.shape

torch.Size([32, 100])